In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/netflix-original-films-imdb-scores/NetflixOriginals.csv")

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data['Premiere'].unique()

In [ ]:
data.columns

In [ ]:
cat = ['Title', 'Genre', 'Language']

In [ ]:
num = ['Runtime']

In [ ]:
y = data['IMDB Score']

In [ ]:
del data['IMDB Score']

In [ ]:
cat_data= pd.get_dummies(data[cat], drop_first=True)

In [ ]:
df = pd.concat([cat_data, data[num]], axis=1)

In [ ]:
df.shape

In [ ]:
x = df.copy()

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_sc = sc.fit_transform(x)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_sc,y, test_size=0.2, random_state=0)

In [ ]:
y_train.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RF = RandomForestRegressor(n_estimators = 100, random_state = 0)
RF.fit(x_train, y_train)

In [ ]:
y_pred = RF.predict(x_test)

In [ ]:
RF.score(x_test, y_test)

In [ ]:
RF.score(x_train, y_train)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
RF = RandomForestRegressor()
RF_Random = RandomizedSearchCV(estimator = RF, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
RF_Random.fit(x_train, y_train)

In [ ]:
RF_Random.best_params_

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [ ]:
best_model = RandomForestRegressor(n_estimators = 1600, random_state = 42)
best_model.fit(x_train, y_train)
best_accuracy = evaluate(best_model, x_test, y_test)

In [ ]:
y_pred = best_model.predict(x_test)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(pd.Series(y_test).sort_values(ascending=True),pd.Series(y_pred).sort_values(ascending=True))
plt.title("Actual vs Predicted")
plt.xlabel("Actual IMDB")
plt.ylabel("Predicted IMDB")
plt.show()